In [1]:
library(extrafont)
library(tidyverse)
library(epidemia)
library(arrow)

Registering fonts with R

── Attaching core tidyverse packages ───────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ─────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp




# 1. parameters

In [2]:
T = 60
T0 = 40

## observation parameters

In [3]:
alpha = 0.01
r_Y = 10

$$Y_t = NegBinom( \frac{EY_t}{r_Y + EY_t}, r_Y )$$

$$ EY_t = \alpha\sum_{s=1}^{t-1} I_s \pi_{t-s} $$

## latent parameters

In [4]:
d = 2
beta = c(1, -3.2)

In [5]:
r_I = 100
K = 6.5

$$I_t = NegBinom( \frac{EI_t}{r_I + EI_t}, r_I )$$

$$ EI_t = R(\bar{A}_t, \beta) \sum_{s=1}^{t-1} I_s g_{t-s} $$

$$R(\bar{A}_t, \beta) = \frac{K}{1+\exp(- \beta^\top A_t)}$$

## seed values

In [6]:
mu = log(100)

$$I_{-T_0} = \mu_0$$

# 2. fit

In [7]:
num_sim = 1000

In [8]:
data <- read_feather('data/simulation_halflong_4.feather')

In [9]:
head(data)

sim,date,R,EI,infection,EY,death,intervention
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
0,0,4.751881,475.1848,516,0.9560691,1,0
0,1,4.751881,511.4127,505,0.9625454,1,0
0,2,4.751881,642.0940,669,0.9683539,1,0
0,3,4.751881,854.3569,836,0.9741112,3,0
0,4,4.751881,1148.1696,942,0.9815892,1,0
0,5,4.751881,1517.5377,1637,0.9935148,3,0


In [13]:
rt <- epirt(formula = R(country, date) ~ 1 + intervention,
            prior = shifted_gamma(shape = 1/6, scale = 1, shift = log(1.05)/6),
            link = scaled_logit(6.5))

In [14]:
inf <- epiinf(gen = EuropeCovid$si, seed_days = T0)

In [15]:
deaths <- epiobs(formula = death ~ 1, i2o = EuropeCovid2$inf2death,
                 prior_intercept = normal(0, 0.2), link = scaled_logit(0.02))

Warning message:
“i2o does not sum to one. Please ensure this is intentional.”


In [16]:
results = array(0, c(num_sim, 6))
colnames(results)=c('b[0]','b[1]','V[0,0]','V[1,0]','V[0,1]','V[1,1]')

In [17]:
for(iter_sim in 1:num_sim){
    start_sim = proc.time()[3]
    
    data_raw = data %>% filter(sim == iter_sim-1)
    data_i = data.frame(country = factor(rep(1,T+T0)),
                        date = as.Date("2022-01-01")
                               + c(-T0:-1, data_raw$date),
                        death = c(rep(NA,T0), data_raw$death),
                        intervention = c(rep(0,T0), data_raw$intervention))
    
    args <- list(rt = rt, inf = inf, obs = deaths, data = data_i, 
                 seed=12345, refresh=0)
    args$algorithm <- "fullrank"; args$iter <- 5e4; args$tol_rel_obj <- 1e-3
    
    fm <- do.call(epim, args)
    
    results[iter_sim,1:2] = colMeans(as.data.frame(fm$stanfit)[1:2])
    results[iter_sim,3:6] = cov(as.data.frame(fm$stanfit)[1:2])
    
    print(paste0(iter_sim,"-th simulation finished, ",
                 proc.time()[3] - start_sim," sec."))
    flush(stdout())
}

[1] "1-th simulation finished, 3.506 sec."


Warning message:
“Pareto k diagnostic value is 0.93. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "2-th simulation finished, 4.926 sec."


Warning message:
“Pareto k diagnostic value is 0.85. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "3-th simulation finished, 4.70399999999999 sec."


...

[1] "990-th simulation finished, 4.5619999999999 sec."
[1] "991-th simulation finished, 2.07099999999991 sec."


Warning message:
“Pareto k diagnostic value is 0.88. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "992-th simulation finished, 4.52799999999979 sec."
[1] "993-th simulation finished, 2.11499999999978 sec."
[1] "994-th simulation finished, 2.04400000000032 sec."
[1] "995-th simulation finished, 4.51399999999967 sec."
[1] "996-th simulation finished, 2.11799999999994 sec."


Warning message:
“Pareto k diagnostic value is 0.83. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "997-th simulation finished, 4.61500000000024 sec."
[1] "998-th simulation finished, 3.20200000000023 sec."
[1] "999-th simulation finished, 4.65099999999984 sec."
[1] "1000-th simulation finished, 3.20800000000008 sec."


In [18]:
write_feather(as.data.frame(results), 
              'coverage_epidemia_nbinom_halflong_4.feather')